# OIKAN Classification Benchmark Tests

This notebook evaluates the OIKANClassifier on various classification tasks to assess:
1. Classification metrics (Accuracy, F1-score, ROC-AUC)
2. Decision boundary interpretability
3. Symbolic formula extraction quality
4. Comparison with traditional classifiers

## Setup and Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

!pip install -qU oikan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.


In [2]:
!pip freeze | grep oikan

oikan==0.0.2.5


In [3]:
import numpy as np
import pandas as pd
from time import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    roc_auc_score,
    f1_score
)
from sklearn.datasets import (
    make_classification,
    make_moons,
    make_circles,
    load_iris,
    load_breast_cancer
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from oikan.model import OIKANClassifier

np.random.seed(42)

## 1. Synthetic Classification Tests

First, let's evaluate OIKAN on synthetic datasets with known decision boundaries.

In [4]:
def generate_synthetic_datasets():
    datasets = {
        'Linear': make_classification(
            n_samples=1000, n_features=2, n_redundant=0,
            n_informative=2, random_state=42,
            n_clusters_per_class=1
        ),
        'Moons': make_moons(
            n_samples=1000, noise=0.1, random_state=42
        ),
        'Circles': make_circles(
            n_samples=1000, noise=0.1, factor=0.3, random_state=42
        )
    }
    return datasets

synthetic_datasets = generate_synthetic_datasets()

In [5]:
def benchmark_classifier(model, X, y, model_name="OIKAN"):
    results = {}
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )
    
    # Training time
    start_time = time()
    model.fit(X_train, y_train)
    train_time = time() - start_time
    
    # Prediction time
    start_time = time()
    y_pred = model.predict(X_test)
    predict_time = time() - start_time
    
    # Metrics
    results = {
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred, average='weighted'),
        'Train Time': train_time,
        'Predict Time': predict_time,
        'Report': classification_report(y_test, y_pred)
    }
    
    if model_name == 'OIKAN':
        symbolic_pred = model.symbolic_predict(X_test)
        results['Symbolic Accuracy'] = accuracy_score(y_test, symbolic_pred)
        results['Symbolic Formula'] = model.get_symbolic_formula()
    
    return results

In [6]:
def run_synthetic_benchmarks():
    results = []
    models = {
        'OIKAN': OIKANClassifier(hidden_dims=[32, 16]),
        'MLPClassifier': MLPClassifier(hidden_layer_sizes=(32, 16), max_iter=500),
        'RandomForest': RandomForestClassifier(n_estimators=100),
        'LogisticRegression': LogisticRegression(),
        'DecisionTree': DecisionTreeClassifier(),
        'SVM': SVC(probability=True)
    }
    
    metrics = ['accuracy', 'roc_auc', 'train_time', 'predict_time']
    
    for dataset_name, (X, y) in synthetic_datasets.items():
        print(f"\nBenchmarking {dataset_name} dataset...")
        
        for model_name, model in models.items():
            res = benchmark_classifier(model, X, y, model_name)
            res['Dataset'] = dataset_name
            results.append(res)
            print(f"{model_name}:")
            print(f"Accuracy = {res['Accuracy']:.4f}")
            if 'ROC AUC' in res:
                print(f"ROC AUC = {res['ROC AUC']:.4f}")
            
            if model_name == 'OIKAN':
                print(f"Symbolic Accuracy = {res['Symbolic Accuracy']:.4f}")
                print("Example decision boundary terms:")
                for i, formula in enumerate(res['Symbolic Formula'][0]):
                    print(f"Class {i}: {formula[:100]}...")
    
    return pd.DataFrame(results)

benchmark_results = run_synthetic_benchmarks()


Benchmarking Linear dataset...
Epoch [10/100], Loss: 0.3077
Epoch [20/100], Loss: 0.1723
Epoch [30/100], Loss: 0.1433
Epoch [40/100], Loss: 0.1263
Epoch [50/100], Loss: 0.1137
Epoch [60/100], Loss: 0.1325
Epoch [70/100], Loss: 0.1238
Epoch [80/100], Loss: 0.1205
Epoch [90/100], Loss: 0.1246
Epoch [100/100], Loss: 0.1253
OIKAN:
Accuracy = 0.9300
Symbolic Accuracy = 0.7200
Example decision boundary terms:
Class 0: 0.1547*x + -0.1855*x^2 + -0.1694*sin(x) + 0.0091*tanh(x) + 0.0023...
Class 1: -0.4246*x + 0.2756*x^2 + 0.1787*sin(x) + 0.0329*tanh(x) + -0.0099...
MLPClassifier:
Accuracy = 0.9350
RandomForest:
Accuracy = 0.9250
LogisticRegression:
Accuracy = 0.9000
DecisionTree:
Accuracy = 0.9200
SVM:
Accuracy = 0.9400

Benchmarking Moons dataset...
Epoch [10/100], Loss: 1.1360
Epoch [20/100], Loss: 0.3949
Epoch [30/100], Loss: 0.2346
Epoch [40/100], Loss: 0.1092
Epoch [50/100], Loss: 0.0540
Epoch [60/100], Loss: 0.0371
Epoch [70/100], Loss: 0.0221
Epoch [80/100], Loss: 0.0182
Epoch [90/100],

## 2. Real Dataset Tests

Now let's evaluate OIKAN on real-world classification problems.

In [7]:
def load_real_datasets():
    datasets = {
        'Iris': load_iris(return_X_y=True),
        'Breast Cancer': load_breast_cancer(return_X_y=True)
    }
    return datasets

real_datasets = load_real_datasets()
real_results = []

for dataset_name, (X, y) in real_datasets.items():
    print(f"\nBenchmarking {dataset_name} dataset...")
    
    # Scale the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Run benchmarks
    oikan = OIKANClassifier(hidden_dims=[64, 32])
    res = benchmark_classifier(oikan, X_scaled, y, 'OIKAN')
    res['Dataset'] = dataset_name
    real_results.append(res)
    
    print(f"OIKAN Accuracy = {res['Accuracy']:.4f}")
    print(f"Symbolic Accuracy = {res['Symbolic Accuracy']:.4f}")
    print("\nExample decision boundary formula:")
    print(res['Symbolic Formula'][0][0])


Benchmarking Iris dataset...
Epoch [10/100], Loss: 0.1565
Epoch [20/100], Loss: 0.0865
Epoch [30/100], Loss: 0.0360
Epoch [40/100], Loss: 0.0640
Epoch [50/100], Loss: 0.0360
Epoch [60/100], Loss: 0.0530
Epoch [70/100], Loss: 0.0140
Epoch [80/100], Loss: 0.0303
Epoch [90/100], Loss: 0.0367
Epoch [100/100], Loss: 0.0237
OIKAN Accuracy = 0.9667
Symbolic Accuracy = 0.5667

Example decision boundary formula:
0.0060*x + -0.0475*x^2 + -0.0880*sin(x) + 0.1535*tanh(x) + -0.0001

Benchmarking Breast Cancer dataset...
Epoch [10/100], Loss: 0.1975
Epoch [20/100], Loss: 0.0712
Epoch [30/100], Loss: 0.0490
Epoch [40/100], Loss: 0.0346
Epoch [50/100], Loss: 0.0182
Epoch [60/100], Loss: 0.0135
Epoch [70/100], Loss: 0.0058
Epoch [80/100], Loss: 0.0108
Epoch [90/100], Loss: 0.0040
Epoch [100/100], Loss: 0.0022
OIKAN Accuracy = 0.9561
Symbolic Accuracy = 0.2105

Example decision boundary formula:
0.0416*x + 0.0332*x^2 + 0.0326*sin(x) + -0.1251*tanh(x) + -0.0000


## 3. Results Analysis

Let's analyze the classification performance metrics in a clear tabular format.

In [8]:
def display_benchmark_results(synthetic_results, real_results):
    # Combine all results
    all_results = pd.concat([pd.DataFrame(synthetic_results), pd.DataFrame(real_results)])
    
    # Format results table with consistent columns
    summary = all_results.pivot_table(
        index=['Dataset', 'Model'],
        values=['Accuracy', 'F1', 'Train Time', 'Predict Time'],
        aggfunc='mean'
    ).round(4)
    
    # Sort by dataset and accuracy
    summary = summary.sort_index(level=0)
    
    print("\nClassification Benchmark Results:")
    print("==============================\n")
    print(summary)
    
    # For OIKAN models, show symbolic accuracy
    oikan_results = all_results[all_results['Model'] == 'OIKAN']
    print("\nOIKAN Symbolic Formula Performance:")
    print("================================\n")
    for _, row in oikan_results.iterrows():
        print(f"Dataset: {row['Dataset']}")
        print(f"Neural Accuracy = {row['Accuracy']:.4f}")
        print(f"Symbolic Accuracy = {row['Symbolic Accuracy']:.4f}\n")

display_benchmark_results(benchmark_results, pd.DataFrame(real_results))


Classification Benchmark Results:

                                  Accuracy      F1  Predict Time  Train Time
Dataset       Model                                                         
Breast Cancer OIKAN                 0.9561  0.9562        0.4120    192.0123
Circles       DecisionTree          0.9750  0.9750        0.0002      0.0019
              LogisticRegression    0.4850  0.4828        0.0002      0.0025
              MLPClassifier         1.0000  1.0000        0.0004      0.4774
              OIKAN                 1.0000  1.0000        0.0866     31.9362
              RandomForest          0.9800  0.9800        0.0060      0.1671
              SVM                   1.0000  1.0000        0.0007      0.0107
Iris          OIKAN                 0.9667  0.9664        0.2383    113.9788
Linear        DecisionTree          0.9200  0.9200        0.0002      0.0025
              LogisticRegression    0.9000  0.8998        0.0003      0.0350
              MLPClassifier         0.93